# КМ-01, Іваник Юрій, Лаб 4

### Імпортуємо бібліотеки

In [5]:
import numpy as np
import tensorflow as tf
from keras import layers, models
from keras.optimizers import SGD 
from keras.datasets import cifar10
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

### Завантажимо дані

In [6]:
batch_size = 32

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train / 255.0 
X_test  = X_test / 255.0 

y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test,  10)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

print(f'Кількість записів у навчальному наборі:   {len(X_train)}')
print(f'Кількість записів у тестовому наборі:     {len(X_test)}')
print(f'Кількість записів у валідаційному наборі: {len(X_val)}')

Кількість записів у навчальному наборі:   50000
Кількість записів у тестовому наборі:     5000
Кількість записів у валідаційному наборі: 5000


### Створимо структуру моделі CNN

In [7]:
model = models.Sequential()

# Додавання згорткових шарів та шарів пулінгу
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))  # Додаємо Dropout

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))  # Додаємо Dropout

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Додавання повнозв'язних шарів
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))  # Додаємо Dropout
model.add(layers.Dense(10, activation='softmax'))  # 10 класів виводу

# Компіляція моделі
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Навчаємо модель

In [8]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_accuracy') is not None and logs.get('val_accuracy') > 0.7:
            self.model.stop_training = True   
            print(f'\n\nAccuracy моделі на валідаційних даних > 70 %, зупинка навчання після епохи № {epoch+1}')


callbacks = myCallback()
try:
    model = models.load_model('my_model.keras')
except:
    history = model.fit(X_train, y_train, epochs=25, batch_size=batch_size, validation_data=(X_val, y_val),
                        callbacks=[callbacks])
    model.save('my_model.keras') # Зберігаємо модель

Epoch 1/25
1563/1563 [==============================] - 61s 37ms/step - loss: 1.8561 - accuracy: 0.3091 - val_loss: 1.5019 - val_accuracy: 0.4470
Epoch 2/25
1563/1563 [==============================] - 57s 37ms/step - loss: 1.5387 - accuracy: 0.4395 - val_loss: 1.3889 - val_accuracy: 0.5162
Epoch 3/25
1563/1563 [==============================] - 55s 35ms/step - loss: 1.4094 - accuracy: 0.4912 - val_loss: 1.3174 - val_accuracy: 0.5264
Epoch 4/25
1563/1563 [==============================] - 55s 35ms/step - loss: 1.3311 - accuracy: 0.5239 - val_loss: 1.2609 - val_accuracy: 0.5534
Epoch 5/25
1563/1563 [==============================] - 56s 36ms/step - loss: 1.2753 - accuracy: 0.5482 - val_loss: 1.1930 - val_accuracy: 0.5766
Epoch 6/25
1563/1563 [==============================] - 58s 37ms/step - loss: 1.2253 - accuracy: 0.5695 - val_loss: 1.1991 - val_accuracy: 0.5770
Epoch 7/25
1563/1563 [==============================] - 61s 39ms/step - loss: 1.1899 - accuracy: 0.5839 - val_loss: 1.1125 -

### Тестуємо модель та знаходимо метрики

In [9]:
test_results = model.evaluate(X_test, y_test, verbose=0)

test_accuracy = test_results[1]
predictions = np.argmax(model.predict(X_test, verbose=0), axis=1)
test_precision = precision_score(np.argmax(y_test, axis=1), predictions, average='weighted')
test_recall = recall_score(np.argmax(y_test, axis=1), predictions, average='weighted')
test_f1_score = f1_score(np.argmax(y_test, axis=1), predictions, average='weighted')

print(f'Accuracy  моделі на тестових даних: {test_accuracy * 100:.2f} %')
print(f'Precision моделі на тестових даних: {test_precision * 100:.2f} %')
print(f'Recall    моделі на тестових даних: {test_recall * 100:.2f} %')
print(f'F1-score  моделі на тестових даних: {test_f1_score * 100:.2f} %')

Accuracy  моделі на тестових даних: 67.74 %
Precision моделі на тестових даних: 69.33 %
Recall    моделі на тестових даних: 67.74 %
F1-score  моделі на тестових даних: 67.42 %
